In [1]:
import cv2
import numpy as np
import glob
import pygame
import os
from keras.models import load_model
try:
    os.mkdir('output_images')
except:
    pass
sift=cv2.xfeatures2d.SIFT_create()
def allign(img_querry,img_train):
    kp_query,desc_query=sift.detectAndCompute(img_querry,None)
    kp_train,desc_train=sift.detectAndCompute(img_train,None)
    index_params = dict(algorithm=0, trees=5)# the indexes that we will be searching in
    search_params = dict()# the search parameters of our search
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(desc_query, desc_train, k=2)# try to find the matches between the decriptors in he first img  and the second one
    good=[]
    for m,n in matches:
        if m.distance <0.75*n.distance:
            good.append(m)
    matches=cv2.drawMatches(img_querry,kp_query,img_train,kp_train,good,None)
    #cv2.imshow("matches",matches)
    q_p=np.float32([kp_query[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    t_p=np.float32([kp_train[m.trainIdx].pt for m in good]).reshape(-1,1,2)
    try:
        matrix,mask=cv2.findHomography(q_p,t_p,cv2.RANSAC)
        alligned=cv2.warpPerspective(img_querry,matrix,(img_train.shape[1],img_train.shape[0]))
    except:
        try:
            matrix=cv2.getPerspectiveTransform(q_p,t_p)
            alligned=cv2.warpPerspective(img_querry,matrix,(img_train.shape[1],img_train.shape[0]))

        except:
            matrix=cv2.getAffineTransform(q_p,t_p)
            alligned=cv2.warpAffine(img_querry,matrix,(img_train.shape[1],img_train.shape[0]))

            
    return alligned

def compare(img_1,img_2):
    delta=cv2.subtract(img_1,img_2)# 1-2
    delta_2=cv2.subtract(img_2,img_1)#2-1
    gray=cv2.cvtColor(delta,cv2.COLOR_BGR2GRAY)
    gray_2=cv2.cvtColor(delta_2,cv2.COLOR_BGR2GRAY)
    blured=cv2.GaussianBlur(gray,(5,5),0)# src,kernel_size,standard deviation
    blured_2=cv2.GaussianBlur(gray_2,(5,5),0)
    _,mask=cv2.threshold(blured,80,255,cv2.THRESH_BINARY)
    _,mask_2=cv2.threshold(blured_2,80,255,cv2.THRESH_BINARY)
    erosion = cv2.erode(mask,None,iterations = 1)
    erosion_2=cv2.erode(mask_2,None,iterations=1)
    cnts=cv2.findContours(erosion,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[-2]# cnts approximaion method
    cnts_2=cv2.findContours(erosion_2,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[-2]
    
    for cnt in cnts:
        area=cv2.contourArea(cnt)
        if area>200:
            cv2.drawContours(img_2,cnt,-1,(255,0,0),2)# -1 stands for cnt index
    for cnt in cnts_2:
        area=cv2.contourArea(cnt)
        if area>200:
            cv2.drawContours(img_2,cnt,-1,(0,0,255),2)# -1 stands for cnt index    
    return mask,mask_2




imgs_dir=glob.glob('images_to_be_detected//*')
model=load_model('classfier_binary_3.h5')
states=['Normal','Infected']
for i in range(len(imgs_dir)-1):
    img=cv2.imread(imgs_dir[i])
    img_1=cv2.imread(imgs_dir[i+1])
    img_new=allign(img_1,img)
    diff,diff_2=compare(img,img_new)
    #cv2.imshow('diff_2',diff_2)
    #cv2.imshow('diff',diff)
    img_o_k=np.array((cv2.resize(img,(256,256)))/(255.0)).reshape(-1,256,256,3)
    img_n_k=np.array((cv2.resize(img_new,(256,256)))/(255.0)).reshape(-1,256,256,3)
    
    old_state=model.predict(img_o_k)
    new_state=model.predict(img_n_k)
    if old_state>=0.5:
        old_state=1
    elif old_state<0.5:
        old_state=0
        
    if new_state>=0.5:
        new_state=1
    elif new_state<0.5:
        new_state=0
    
    #cv2.imshow('old',img)
    #cv2.imshow('img_1',img_1)
    #cv2.imshow('new',img_new)
    conc=np.concatenate((img,img_new),axis=1)
    cv2.line(conc,(conc.shape[1]//2,0),(conc.shape[1]//2,conc.shape[0]),(0,255,0),2)
    cv2.putText(conc,'Old image',(20,30),cv2.FONT_HERSHEY_COMPLEX,0.9,(255,255,0),2)
    cv2.putText(conc,'New image',(conc.shape[1]-190,30),cv2.FONT_HERSHEY_COMPLEX,0.9,(255,255,0),2)
    
    cv2.putText(conc,states[old_state],(20,60),cv2.FONT_HERSHEY_COMPLEX,0.9,(255,255,0),2)
    cv2.putText(conc,states[new_state],(conc.shape[1]-190,60),cv2.FONT_HERSHEY_COMPLEX,0.9,(255,255,0),2)
    cv2.imwrite(f'output_images//{i}.jpg',conc)
    #cv2.imshow('conc',conc)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    
    
    
    
    
class GUI:
    def __init__(self):
        self.width=1200
        self.height=800
        self.white=(255,255,255)
        self.fps=60
        self.to_show_index=0
        self.win=pygame.display.set_mode((self.width,self.height))
        self.run=True
        pygame.display.set_caption('Change detection from CT_scan for COVID-19')
        self.directory=glob.glob(r'output_images//*')
        self.clock=pygame.time.Clock()
        self.imgs=[]
        for i in self.directory:
            self.imgs.append(pygame.transform.scale(pygame.image.load(i),(self.width,self.height)))
        
    def draw(self):
        self.win.fill(self.white)
        self.win.blit(self.imgs[self.to_show_index],(0,0))
        pygame.display.update()
    def main(self):
        while self.run:
            for event in pygame.event.get():
                if event.type ==pygame.QUIT:
                    self.run=False
                    pygame.quit()
                    return
                if event.type==pygame.KEYDOWN:
                    if event.key==pygame.K_RIGHT:
                        if self.to_show_index==len(self.directory)-1:
                            self.to_show_index=0
                        else:
                            self.to_show_index+=1
                    if event.key ==pygame.K_LEFT:
                        if self.to_show_index==0:
                            self.to_show_index=len(self.directory)-1
                        else:
                            self.to_show_index-=1
            self.draw()

G=GUI()
G.main()        
            
    

    

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.
E:\conda\envs\iris\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\conda\envs\iris\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\conda\envs\iris\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\conda\envs\iris\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Pas

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
